# Coding the Binary Tree Model for pricing Options and the Underlying

In [29]:
import numpy as np
from scipy import stats

First, create the `Node` class which we will use to generate the binary tree representing the evolution of the price of the underlying.

In [24]:
class Node_Underlying:
    def __init__(self, val, prob, left=None, right=None):
        self.val = val
        self.prob = prob
        self.left = left
        self.right = right

    def is_leaf(self):
        return self.left is None and self.right is None
    
def collect_leaves(tree):
    if tree.is_leaf():
        return np.array([[tree.val, tree.prob]])
    else:
        return np.concat((collect_leaves(tree.left), collect_leaves(tree.right)), axis=0)

## Binary Tree for the Underlying

Now, we want to create the binary tree for the value evolution of the underlying by prescribing a mean $\mu$ a volatility parameter $\sigma$ and a time interval $\delta t$.

In [100]:
def create_binary_underlying(depth: int, current_val: float, drift: float, vol: float, dt: float, current_prob: float=1.0, current_depth: int=0):
    if 0.5 + drift * np.sqrt(dt) / (2 * vol) > 1:
        raise ValueError("The parameters are chosen s.t. probability p for rise is p > 1.")
    if current_depth == depth + 1:
        return
    else:
        rising_node = create_binary_underlying(depth, current_val * (1 + vol * np.sqrt(dt)), drift, vol, dt, current_prob * (1/2 + drift * np.sqrt(dt) / (2 * vol)), current_depth + 1)
        falling_node = create_binary_underlying(depth, current_val * (1 - vol * np.sqrt(dt)), drift, vol, dt, current_prob * (1/2 - drift * np.sqrt(dt) / (2 * vol)), current_depth + 1)
        root_node = Node_Underlying(current_val, current_prob, rising_node, falling_node)
        return root_node

def take_path(tree, drift: float, vol: float, dt: float):
    if tree.is_leaf():
        return np.array([tree.val])
     
    prob = 1/2 + drift * np.sqrt(dt) / (2 * vol)
    X = np.random.binomial(1, prob)
    if X == 1:
        return np.concat(([tree.val], take_path(tree.left, drift, vol, dt)))
    elif X == 0:
        return np.concat(([tree.val], take_path(tree.right, drift, vol, dt)))
    
def expected_underlying_price(tree):
    leaves = collect_leaves(tree)
    return np.sum(leaves[:,0] * leaves[:,1])

In [101]:
tree = create_binary_underlying(20, 100.0, 0.1, 0.01, 0.01)
expected_underlying_price(tree)

np.float64(102.01911448605382)

Creating the binary tree clearly scales exponentially as there are $2^N$ leaf nodes for a depth $N$. So instead of calculating an entire tree and then taking random paths in this tree according to the probability $p$, we could also create different realizations without the whole tree.

In [116]:
def generate_path(init_val: float, drift: float, vol: float, dt: float, depth: int):
    current_val = init_val
    vals = np.array([current_val])
    u = 1 + vol * np.sqrt(dt)
    v = 1 - vol * np.sqrt(dt)
    p = 1/2 + drift * np.sqrt(dt) / (2 * vol)
    for i in range(depth):
        X = np.random.binomial(1, p)
        if X == 1:
            current_val = current_val * u
        if X == 0:
            current_val = current_val * v
        vals = np.append(vals, current_val)
    return vals

generate_path(100.0,  0.01, 0.01, 0.01, 100)

array([100.        , 100.1       , 100.2001    , 100.3003001 ,
       100.1999998 , 100.3001998 , 100.1998996 , 100.3000995 ,
       100.1997994 , 100.2999992 , 100.1996992 , 100.2998989 ,
       100.199599  , 100.0993994 , 100.1994988 , 100.0992993 ,
       100.1993986 , 100.0991992 , 100.1992984 , 100.2994977 ,
       100.1991982 , 100.2993974 , 100.3996968 , 100.5000965 ,
       100.3995964 , 100.2991968 , 100.399496  , 100.2990965 ,
       100.3993956 , 100.2989962 , 100.19869721, 100.09849851,
        99.99840001, 100.09839841,  99.99830001,  99.89830171,
        99.79840341,  99.69860501,  99.79830361,  99.69850531,
        99.5988068 ,  99.69840561,  99.59870721,  99.69830591,
        99.59860761,  99.499009  ,  99.39950999,  99.4989095 ,
        99.59840841,  99.49881   ,  99.59830881,  99.69790712,
        99.79760503,  99.89740263,  99.79750523,  99.89730274,
        99.99720004, 100.09719724,  99.99710004, 100.09709714,
        99.99700004, 100.09699704, 100.19709404, 100.29